## 포함된 내용 - 추가 칼럼 생성

* job1 - job 칼럼 1차 전처리(괄호 제거) 완료, 2차 전처리 월요일 이후 진행 예정
* newbie - 신입/경력 구분
* degree - 학사/석사 구분
* hard_skill - 자격요건에서 요구하는 프로그래밍 언어, 라이브러리, 프레임워크, 문서작업 툴 

#### 대부분 파이썬 정규표현식 사용

In [1]:
import pandas as pd
import re

In [2]:
## search로 분류된 + 중복 제거된 데이터를 가져와서 사용

### 중복제거 시, 기존에는 url만 중복되는 것만 제거했는데 
### 하다보니 url은 달라도 내용이 완전 똑같은 공고가 여럿 있어서 이것도 함께 제거한 데이터를 만들어 사용함

df = pd.read_excel('mildsix_1021.xlsx')
df.head()

,company,job,company_intro,work,job_require,preference,url,search
0,악어디지털,딥러닝모델개발자,회사소개 악어디지털은 지난 2014년 문서 전자화 O2O 서비스 제공 기업으...,데이터분석 딥러닝 모델 훈련 및 모니터링 데이터 파이프라인 시스템 설계 ...,2년 이상 경력자 python을 이용한 프로그램 작성 경험 딥러닝 프레임...,OCR 데이터 활용 경험 스캔된 문서 처리 경험,https://www.wanted.co.kr/wd/132626,DE
1,수성엔지니어링,데이터 엔지니어 (Python specialist),수성엔지니어링 기술연구소 개발팀 소개 수성엔지니어링은 한국 엔지니어링 업계 TO...,Python 기반으로 공간데이터 처리 알고리즘 개발 Docker AWS ECS...,Python을 능숙하게 다룰 수 있는 개발자 객체지향 프로그래밍에 대한 충분...,Slack Notion GitHub등 협업툴 사용에 익숙하신 분 No...,https://www.wanted.co.kr/wd/103169,DE
2,아인시스에이아이랩,Data Engineer,IT 비즈니스 혁신을 선도하는 AI 스타트업 아인시스에이아이랩 이하 에이아이랩 ...,데이터 분석을 위한 알고리즘 및 개발 방향과 범위를 산정해요 데이터 수집 및...,이공계 졸업 예정자 혹은 관련 경력을 보유하신 분 네트워크 애플리케이션 개...,스타트업 업무경험이 있으신 분 Rust 개발 경험이 있으신 분 머신러닝 ...,https://www.wanted.co.kr/wd/130687,DE
3,유진로봇,개발본부 AMS팀 백엔드(Back-end) 개발자,Innovation for Quality of Life with Intellige...,서비스 운영에 필요한 로봇관리를 위한 소프트웨어 설계 및 개발 리소스 관리...,Javascript ES6 ES7 사용자 Database 사용 가능자...,리눅스 환경에서의 개발 익숙자 RESTful API 설계 경험자 Ne...,https://www.wanted.co.kr/wd/126811,DE
4,스콥정보통신,"AI(인공지능), 빅데이터 개발자",스콥정보통신은 1999년 설립 이후 RMON 기술을 이용한 네트워크 트래픽 ...,빅데이터 AI 데이터 엔지니어링 분석 및 처리 데이터 파이프라인 개발과 ...,Tensorflow PyTorch Keras 등 머신러닝 딥러닝 프레임워크 ...,로그시스템 및 데이터마트 Structure 설계 경험 Hadoop MR ...,https://www.wanted.co.kr/wd/129440,DE


In [3]:
## 대문자 소문자가 마구 섞여서 이를 통일해 줄 필요가 있음
### 일단 url 제외하고 모두 소문자로 바꾸기

cols = ['company', 'job', 'company_intro', 'work', 'job_require', 'preference']
for col in cols :
    df[col] = df[col].apply(str.lower)  

## 신입/경력

In [4]:
## 신입/경력 구분
### 직무명에 신입 있거나 자격요건에 신입 포함되면 신입으로 분류

df['newbie'] = ''
for i in range(len(df)):
    if re.search('.*신입.*', df['job'].iloc[i]) or re.search('.*신입.*', df['job_require'].iloc[i]):
        df['newbie'].iloc[i] = '신입'
    else:
        df['newbie'].iloc[i] = '경력'

df.newbie.value_counts()

경력    1117
신입      50
Name: newbie, dtype: int64

## 학력 - 석사/학사

In [5]:
## 학력 - 석사 이상 요구 1, 아니면 0

df['degree'] = ''
for i in range(len(df)):
    if not re.search('석사.*|박사.*', df['job_require'].iloc[i]):
        df['degree'].iloc[i] = 0
    else:
        df['degree'].iloc[i] = 1

df.degree.value_counts()

0    1119
1      48
Name: degree, dtype: int64

## 직무명(job) 1차 전처리

In [6]:
## 직무명(job column) - 대괄호, 괄호 내용포함 다 지우기

result = []
for i, v in enumerate(df['job']):
    value = ' '.join(re.split('\[[a-z가-힣0-9].*\]|\([a-z가-힣0-9].*\)|-', v)).strip()
    result.append(value)
    
#print(result)

#df['job1'] = result

In [7]:
### 직무명은 2차로 한 번 더 클렌징 해줄 예정

### 이 때, 어디까지를 비슷한 이름으로 묶을 것인지 논의해야 함

## 하드스킬 추출 

In [8]:
## 하드스킬 - 프로그래밍 언어, 라이브러리, 프레임워크... 따로 뽑아서 정리

## 일단 하드스킬만 보기위해 어떻게 하면 좋을까..
### 대부분 하드스킬은 영어, 일단 영어로 된 걸 가져와서 맞는지 살펴보자
## 보니까 대충 맞더라. 일단 여기 내에서 하드스킬을 추출해보자

df['hard_skills'] = ''
for i in range(len(df)):
    if  re.search('[a-z].*', df['job_require'].iloc[i]):
        df['hard_skills'].iloc[i] = re.search('[a-z].*', df['job_require'].iloc[i]).group()
    
    ## 하드스킬에 해당되는게 없는 공고는 그냥 'none' 이라고 쓰자 
    else :
        df['hard_skills'].iloc[i] = 'none'
        
df['hard_skills']

0       python을 이용한 프로그램 작성 경험   딥러닝 프레임워크 tensorflow ...
1       python을 능숙하게 다룰 수 있는 개발자   객체지향 프로그래밍에 대한 충분한 ...
2       c   c    java  rust 중 1개 이상 코딩 경험이 있으신 분   nos...
3       javascript es6  es7   사용자    database 사용 가능자  ...
4       tensorflow  pytorch  keras 등 머신러닝 딥러닝 프레임워크 사용...
                              ...                        
1162    c python 프로그래밍 언어에 능숙하며 문제해결이 가능하신 분 rest api ...
1163    c python 프로그래밍 언어에 능숙하며 문제해결이 가능하신 분 rest api ...
1164    linux php mysql sas r python 4 자격 조건 아래 중 1개 이...
1165    java servletjsp 개발경험 필수 javascript css를 이용한 반응...
1166                               ai 모델 개발에 현실 데이터 사용 경험
Name: hard_skills, Length: 1167, dtype: object

In [9]:
## 그다음에 하드스킬 전체 데이터를 쭉 훑어서 많이 등장하는 기술을 하나씩 찾아서 적음
##### 작성 기준은 최대한 영문으로. 띄어쓰기된 영문은 일단 뽑히도록 리스트에는 넣어두고 추후 이름 수정

## manager, gtm, tag  => google tag manager
## ga, analytics => google analytics
## powerpoint => ppt
## tracker => app tracker
## re, dash => redash
## es => elasticsearch


hard_skills = ['python', 'sql', 'r', 'java', 'javascript' 'spring', 'kotlin', 'scala', 'go', 'c', 'es', 'ga', 'amplitude',
        'node', 'nosql', 'spark', 'hadoop', 'kafka', 'hive', 'bi', 'tableau','aws', 'gcp','azura','rdbms',  'html','git','airflow',
        'apache','k8s','elasticsearch', 'linux', 'mysql', 'postgresql','ms', 'matlab', 'kubernetes','docker', 'typescript', 
        'django', 'fastapi', 'flask', 'mariadb', 'rdb', 'bigquery', 'redash', 'studio', 'google', 'spread', 'sheet', 'cloud', 'shell',
       'analytics', 'appsflyer', 'braze', 'tag', 'manager' ,'gtm', 'mongodb', 'jupyter', 'redshift', 'react', 'vue', 'angular', 
       'nodejs', 'mlops', 'devops', 'amp', 'firebase', 'braze', 'saas', 'adjust', 'mmp', 'excel', 'tracker', 'ppt', 
        'figma', 'radish', 'adobe', 'sa','gfa', 'da','mixpanel', 'photoshop', 'illustrator', 'premiere', 'ads', 'airbridge','fixel', 
         'utm', 'slack', 'notion',  'hue', 'quicksight', 'spreadsheet', 'powerpoint', 're', 'dash', 'db', 'zeplin',
        'oa', 'sketch',  'workspace', 'jira', 'trello', 'adobeoxd', 'axure', 'tensorflow', 'pytorch', 'keras',  
        'pandas', 'numpy', 'cpython', 'presto']

In [10]:
### 데이터를 전체적으로 쭉 보면 한글, 영어 다 섞여 있음
### 그 중 자주 나오는 스킬인데 한글로 나오는 건 따로 체크해서 영문으로 변경
### 변경할 내용이 너무 많아져서 이를 사전으로 만들어서 값을 치환(replace)함

need_to_replace = {'파이썬':'python','앰플리튜드':'amplitude', '애널리틱스':'ga', '엑셀':'excel', '엑셀등':'excel', 
                    '앱스플라이어':'appsfyler', '애드저스트':'adjust', '포토샵':'photoshop', '파워포인트':'ppt', 
                    '인텔리전스':'bi', '클라우드':'cloud', '프리젠테이션':'ppt', '일러스트레이터':'illustrator',
                    '프리미어':'premiere'}

for key, val in need_to_replace.items():
    df['job_require'] = df['job_require'].apply(lambda x: x.replace(key, val))

In [11]:
result = []
for i in range(len(df)):
    splited = set(' '.join(re.split('[가-힣0-9]', df['hard_skills'].iloc[i])).split())
    #print(splited)
    sub = []
    for value in splited:
        if value in hard_skills:
            sub.append(value)

    result.append(sub)

print(result)
## 결과에서 비어있는건 별다른 하드스킬을 요구하지 않은 곳임

[['tensorflow', 'python', 'keras', 'mlops'], ['amp', 'aws', 'python', 'git', 'docker'], ['elasticsearch', 'c', 'git', 'java', 'mongodb', 'nosql'], ['node', 'es'], ['tensorflow', 'pytorch', 'python', 'go', 'kotlin', 'java', 'keras', 'scala'], ['java'], ['python', 'c'], [], [], ['sql', 'python'], ['amp'], ['python', 'ga', 'amplitude', 'airflow'], ['node', 'flask', 'django', 'fastapi', 'sql', 'nosql'], ['python', 'airflow', 'java', 'db', 'sql', 'scala'], [], [], ['tensorflow', 'pytorch', 'python', 'shell', 'c', 'java'], ['sql'], ['node', 'mongodb'], ['tensorflow', 'r', 'pytorch', 'pandas', 'python', 'spark', 'sql'], ['aws', 'python', 'go', 'java', 'scala'], ['sql'], ['kafka', 'hive', 'python', 'hadoop', 'presto', 'spark', 'sql'], ['python', 'airflow', 'java', 'db', 'sql', 'scala'], ['spark'], ['jupyter', 'python', 'aws', 'gcp'], ['ms', 'c', 'tableau', 'java', 'sql', 'bi'], ['python'], ['tensorflow', 'pytorch'], ['python', 'go', 'linux', 'java', 'sql', 'scala'], [], ['aws', 'pandas', 'pyth

In [12]:
################### 이제 일부 같은 의미의 단어를 하나로 통일시킬 방법 생각해야 함!########################
## manager, gtm, tag  => google tag manager
## ga, analytics => google analytics
## powerpoint, ppt => ppt
## tracker => app tracker
## re, dash => redash
## es => elasticsearch
## studio => google data studio

In [13]:
## 이것저것 해보고 안되고..(왜 안되지?)
## 일단 아래 방법이 성공했는데, 더 좋은 방법을 찾고싶다....

for x in result:
    for i in range(len(x)):
        x[i] = re.sub('manager', 'gtm', x[i])
        x[i] = re.sub('tag', 'gtm', x[i])
        x[i] = re.sub('powerpoint', 'ppt',x[i])
        x[i] = re.sub('tracker', 'Apptracker',x[i])
        x[i] = re.sub('^re$', 'redash',x[i])
        x[i] = re.sub('dash', 'redash',x[i])
        x[i] = re.sub('^es$', 'elasticsearch',x[i])
        x[i] = re.sub('^analytics', 'ga',x[i])
        x[i] = re.sub('^studio', 'datastudio',x[i])
        
print(result)

[['tensorflow', 'python', 'keras', 'mlops'], ['amp', 'aws', 'python', 'git', 'docker'], ['elasticsearch', 'c', 'git', 'java', 'mongodb', 'nosql'], ['node', 'elasticsearch'], ['tensorflow', 'pytorch', 'python', 'go', 'kotlin', 'java', 'keras', 'scala'], ['java'], ['python', 'c'], [], [], ['sql', 'python'], ['amp'], ['python', 'ga', 'amplitude', 'airflow'], ['node', 'flask', 'django', 'fastapi', 'sql', 'nosql'], ['python', 'airflow', 'java', 'db', 'sql', 'scala'], [], [], ['tensorflow', 'pytorch', 'python', 'shell', 'c', 'java'], ['sql'], ['node', 'mongodb'], ['tensorflow', 'r', 'pytorch', 'pandas', 'python', 'spark', 'sql'], ['aws', 'python', 'go', 'java', 'scala'], ['sql'], ['kafka', 'hive', 'python', 'hadoop', 'presto', 'spark', 'sql'], ['python', 'airflow', 'java', 'db', 'sql', 'scala'], ['spark'], ['jupyter', 'python', 'aws', 'gcp'], ['ms', 'c', 'tableau', 'java', 'sql', 'bi'], ['python'], ['tensorflow', 'pytorch'], ['python', 'go', 'linux', 'java', 'sql', 'scala'], [], ['aws', 'pan

In [14]:
## bi => BI, r -> R, mlops -> MLOps, c -> C, db -> DB, devops -> DevOps, oa -> OA
# gcp -> GCP, ms -> MS, , saas -> SAAS, gfa => GFA, amp -> AMP, nosql -> NoSQL 
## 나머지는 앞글자만 대문자로..

### 그냥 if length가 3 -> 모두 대문자, else : 앞에만 대문자

result1 = []
for x in result:
    sub1 = []
    for v in x:
        if len(v) <= 3:
            v = v.upper()
            sub1.append(v)
        else :
            v = v.capitalize()
            sub1.append(v)
        sub1 = list(set(sub1))
    result1.append(sub1)
print(result1)

[['Tensorflow', 'Python', 'Mlops', 'Keras'], ['Docker', 'GIT', 'AWS', 'Python', 'AMP'], ['Nosql', 'Mongodb', 'GIT', 'C', 'Elasticsearch', 'Java'], ['Elasticsearch', 'Node'], ['Tensorflow', 'Scala', 'Pytorch', 'GO', 'Java', 'Keras', 'Python', 'Kotlin'], ['Java'], ['C', 'Python'], [], [], ['Python', 'SQL'], ['AMP'], ['Airflow', 'Amplitude', 'Python', 'GA'], ['SQL', 'Django', 'Nosql', 'Fastapi', 'Flask', 'Node'], ['SQL', 'DB', 'Scala', 'Airflow', 'Java', 'Python'], [], [], ['Tensorflow', 'Shell', 'C', 'Pytorch', 'Java', 'Python'], ['SQL'], ['Mongodb', 'Node'], ['R', 'Tensorflow', 'SQL', 'Spark', 'Pandas', 'Pytorch', 'Python'], ['Scala', 'GO', 'AWS', 'Java', 'Python'], ['SQL'], ['SQL', 'Spark', 'Hive', 'Kafka', 'Presto', 'Python', 'Hadoop'], ['SQL', 'DB', 'Scala', 'Airflow', 'Java', 'Python'], ['Spark'], ['AWS', 'Python', 'GCP', 'Jupyter'], ['MS', 'BI', 'SQL', 'Tableau', 'C', 'Java'], ['Python'], ['Tensorflow', 'Pytorch'], ['SQL', 'Scala', 'GO', 'Java', 'Python', 'Linux'], [], ['AWS', 'Pan

In [15]:
## 근데 최종 결과물이 리스트로 나와서 일단 문자열로 바뀌서 데이터 프레임에 넣기
new = []
for each in result1:
    new.append(' '.join(each))
#print(new)

df['hard_skill'] = new

In [16]:
## 현재까지 새로 만든 칼럼을 중심으로 보면(job1은 더 다듬어져야 함) 

df2 = df[['company', 'newbie', 'degree','hard_skill']]
df2.head()

,company,newbie,degree,hard_skill
0,악어디지털,경력,0,Tensorflow Python Mlops Keras
1,수성엔지니어링,경력,0,Docker GIT AWS Python AMP
2,아인시스에이아이랩,경력,0,Nosql Mongodb GIT C Elasticsearch Java
3,유진로봇,경력,0,Elasticsearch Node
4,스콥정보통신,경력,0,Tensorflow Scala Pytorch GO Java Keras Python ...


In [17]:
df3 = df[['company', 'job', 'search', 'newbie', 'degree', 'hard_skill',  'work','job_require', 'preference']]
#df3.to_excel('mildsix_1024.xlsx', index = False)

In [18]:
df3.head()

,company,job,search,newbie,degree,hard_skill,work,job_require,preference
0,악어디지털,딥러닝모델개발자,DE,경력,0,Tensorflow Python Mlops Keras,데이터분석 딥러닝 모델 훈련 및 모니터링 데이터 파이프라인 시스템 설계 ...,2년 이상 경력자 python을 이용한 프로그램 작성 경험 딥러닝 프레임...,ocr 데이터 활용 경험 스캔된 문서 처리 경험
1,수성엔지니어링,데이터 엔지니어 (python specialist),DE,경력,0,Docker GIT AWS Python AMP,python 기반으로 공간데이터 처리 알고리즘 개발 docker aws ecs...,python을 능숙하게 다룰 수 있는 개발자 객체지향 프로그래밍에 대한 충분...,slack notion github등 협업툴 사용에 익숙하신 분 no...
2,아인시스에이아이랩,data engineer,DE,경력,0,Nosql Mongodb GIT C Elasticsearch Java,데이터 분석을 위한 알고리즘 및 개발 방향과 범위를 산정해요 데이터 수집 및...,이공계 졸업 예정자 혹은 관련 경력을 보유하신 분 네트워크 애플리케이션 개...,스타트업 업무경험이 있으신 분 rust 개발 경험이 있으신 분 머신러닝 ...
3,유진로봇,개발본부 ams팀 백엔드(back-end) 개발자,DE,경력,0,Elasticsearch Node,서비스 운영에 필요한 로봇관리를 위한 소프트웨어 설계 및 개발 리소스 관리...,javascript es6 es7 사용자 database 사용 가능자...,리눅스 환경에서의 개발 익숙자 restful api 설계 경험자 ne...
4,스콥정보통신,"ai(인공지능), 빅데이터 개발자",DE,경력,0,Tensorflow Scala Pytorch GO Java Keras Python ...,빅데이터 ai 데이터 엔지니어링 분석 및 처리 데이터 파이프라인 개발과 ...,tensorflow pytorch keras 등 머신러닝 딥러닝 프레임워크 ...,로그시스템 및 데이터마트 structure 설계 경험 hadoop mr ...


In [19]:
## 신입인 경우는?
df2[df2.newbie == '신입']

,company,newbie,degree,hard_skill
18,브이티피엘,신입,0,Mongodb Node
34,코드에프,신입,0,Html
56,에듀윌,신입,0,
60,블랙탠저린,신입,1,Python Mlops
95,하이퍼라운지,신입,0,PPT Excel
118,로그프레소,신입,0,
119,로그프레소,신입,1,
165,리얼에이이,신입,0,
175,러닝스푼즈,신입,0,
209,케어마인드,신입,0,


In [20]:
## 석사 이상 요구하는 경우는?
df2[df2.degree == 1]

,company,newbie,degree,hard_skill
10,원티드랩,경력,1,AMP
28,스캐터랩(scatterlab),경력,1,Tensorflow Pytorch
57,누비랩,경력,1,
58,누비랩,경력,1,
60,블랙탠저린,신입,1,Python Mlops
99,데이터헌트 (피스컬노트),경력,1,
106,수퍼톤,경력,1,Tensorflow Pytorch Docker
114,버킷플레이스(오늘의집),경력,1,Cloud
119,로그프레소,신입,1,
453,딥브레인에이아이,경력,1,Python


In [21]:
## (데이터를 가져온) 원본 데이터의 모습은
df[['company', 'job', 'job_require']]

,company,job,job_require
0,악어디지털,딥러닝모델개발자,2년 이상 경력자 python을 이용한 프로그램 작성 경험 딥러닝 프레임...
1,수성엔지니어링,데이터 엔지니어 (python specialist),python을 능숙하게 다룰 수 있는 개발자 객체지향 프로그래밍에 대한 충분...
2,아인시스에이아이랩,data engineer,이공계 졸업 예정자 혹은 관련 경력을 보유하신 분 네트워크 애플리케이션 개...
3,유진로봇,개발본부 ams팀 백엔드(back-end) 개발자,javascript es6 es7 사용자 database 사용 가능자...
4,스콥정보통신,"ai(인공지능), 빅데이터 개발자",tensorflow pytorch keras 등 머신러닝 딥러닝 프레임워크 ...
...,...,...,...
1162,비바엔에스,[잇츠스마트팜/대전] ai 엔지니어 (ai engineer),c python 프로그래밍 언어에 능숙하며 문제해결이 가능하신 분 rest api ...
1163,비바엔에스,[잇츠스마트팜/서울] ai 엔지니어 (ai engineer),c python 프로그래밍 언어에 능숙하며 문제해결이 가능하신 분 rest api ...
1164,핌아시아,data 활용/분석 담당자,1 직급 과장2 경력 4년 이상3 개발언어 linux php mysql sas r ...
1165,넥스다임,연구원,경력3년이상 경력7년이상 java servletjsp 개발경험 필수 javascri...
